CAPÍTULO 9 - Apply
-


Conhecer apply é um conceito fundamental no processo de limpeza de dados. Apply aceita uma função e execulta em cada linha ou coluna do dataframe simultamneamente.

9.3 Apply (básico)

In [2]:
import pandas as pd

df = pd.DataFrame({'a': [10,20,30],
                   'b': [20,30,40]})

print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [4]:
def my_sq(x):
    return x ** 2

Podemos aplicar funções com apply em uma Series(isto é, uma coluna ou linha individual)

In [3]:
print(df['a'] ** 2)

0    100
1    400
2    900
Name: a, dtype: int64


9.3.1 apply em uma Series

Em nosso exemplo, se o subconjunto for composto por uma só coluna ou linha, o seu type será uma Series do Pandas

Series tem um método chamado apply, para usá-lo, devemos passar a função que queremos usar em cada elemento da Series.

In [6]:
sq = df['a'].apply(my_sq)

print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


Observe que não precisamos de parênteses quando passamos a função

Vamos fazer um outro exemplo mas agora com uma função que use dois parâmetros

In [8]:
def my_exp(x, e):
    return x ** e

Se quisermos usar a função devos passar um segundo parâmetro. Para isso basta passar o segundo argumento como uma Keyword Argument (argumento nomeado) para apply:

In [10]:
ex = df['a'].apply(my_exp, e=3)

print(ex)

0     1000
1     8000
2    27000
Name: a, dtype: int64


9.3.2 apply em um DataFrame

In [11]:
print(df)

    a   b
0  10  20
1  20  30
2  30  40


DataFrames têm pelo menos duas dimensões, então devemos expecificar em qual eixo em que a função será aplicada

In [13]:
def printMe(x):
    print(x)

9.3.2.1 Operações em colunas

Use o parâmetro axis=0 em apply para trabalhar com colunas

In [17]:
df.apply(printMe, axis=0)

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64


a    None
b    None
dtype: object

Quando aplicamos uma função em um DataFrame, o eixo todo é passado como primeiro argumento da função

Para ilustrar melhor vamos escrever uma função que calcule a média de três números

In [18]:
def avg3(x, y, z):
    return (x + y + z) / 3

Se tentarmos aplicar essa função em nossar colunas veremos um erro

In [20]:
"""
print(df.apply(avg3))
"""

'\nprint(df.apply(avg3))\n'

A função recebe o primeiro argumento, mas deixa de receber y e z, isso porque quando usamos apply a coluna toda é passada como primeiro argumento

Para que essa função seja apropriada ao método apply, será nescessário reescrever algumas partes

In [25]:
def avg3Apply(cool):
    x = cool[0]
    y = cool[1]
    z = cool[2]

    return (x + y + z) / 3

In [26]:
print(df.apply(avg3Apply))

a    20.0
b    30.0
dtype: float64


9.3.2.2 Operações em linhas

As operações em linhas funcionam como as operações me colunas, com a diferença que agora passamos axis=1,  
ao invés de a primeira coluna interira ser passado como primeiro parâmetro, agora a primeira linha inteira é passada como primeiro parâmetro

No entanto, a função que escrevemos não funcionará para as linhas, poís temos apenas 2 números em uma linha, e a função precisa de 3 para funcionar

In [28]:
"""
print(df.apply(avg3Apply, axis=1))
""" 

'\nprint(df.apply(avg3Apply, axis=1))\n'

In [29]:
def avg2Apply(row):
    x = row[0]
    y = row[1]

    return (x + y) / 2

In [30]:
print(df.apply(avg2Apply, axis=1))

0    15.0
1    25.0
2    35.0
dtype: float64


C:\Users\User\AppData\Local\Temp\ipykernel_23228\2954858172.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = row[0]
C:\Users\User\AppData\Local\Temp\ipykernel_23228\2954858172.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = row[1]


9.4 apply (mais avançado)

A biblioteca seaborn tem um conjunto de dados embutido chamdo titanic que informa se algum indivíduo sobreviveu ao naufrágio do Titanic

In [31]:
import seaborn as sns

titanic = sns.load_dataset("titanic")

In [32]:
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None


Uma forma de usar apply é calcular quatos valores null ou NaN há em nossos dados, assim como o percentual de casos completos em cada coluna ou cada linha.  
Vamos escrever algumas funções

Vec = Vector
-


- Número de valores ausentes

In [33]:
import numpy as np

def count_missing(vec):

    #obtém um vetor com valores True/Flase conforme o valor estiver ausente ou não
    nullVec = pd.isnull(vec)

    #soma todos os valore que forem nulos
    nullCont = sum(nullVec)

    return nullCont

- Proporção de valors ausentes

In [35]:
def prop_missing(vec):

    #numerador: número de valres ausentes
    num = count_missing(vec)

    #denominador: número total de valores em um vetor
    #precisamos contabilizar também os valores ausentes
    dem = vec.size

    return num / dem

- Proporção de valores completos

In [36]:
#porcentagem de valores não ausentes
def prop_complete(vec):

    return 1 - prop_missing(vec)

9.4.1 Operações em coluna 

Vamo aplicar nossas funções me cada coluna dos nossos dados

In [38]:
cmis_col = titanic.apply(count_missing)
pmis_col = titanic.apply(prop_missing)
pcom_col = titanic.apply(prop_complete)

print(cmis_col, '\n')
print(pmis_col, '\n')
print(pcom_col, '\n')

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64 

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64 

survived       1.000000
pclass         1.000000
sex            1.000000
age            0.801347
sibsp          1.000000
parch          1.000000
fare           1.000000
embarked       0.997755
class          1.000000
who            1.000000
adult_male     1.000000
deck           0.227834
embark_town    0.997755
alive      

Como temos contadores de valores ausentes podemos determinar se uma coluna é ou não uma opção viável para análise.  
Exemplo, na coluna embark_town, podemos ver que somente dois valores estão ausentes, e podemos verificar se estão assim aleatoriamente ou se há algum motivo especial para isso

In [39]:
print(titanic.loc[pd.isnull(titanic.embark_town), :])

     survived  pclass     sex   age  sibsp  parch  fare embarked  class  \
61          1       1  female  38.0      0      0  80.0      NaN  First   
829         1       1  female  62.0      0      0  80.0      NaN  First   

       who  adult_male deck embark_town alive  alone  
61   woman       False    B         NaN   yes   True  
829  woman       False    B         NaN   yes   True  


Outra observação é que a variável deck em 688 (77,2%) de valores ausentes. A menos que tenha mais investigações, é seguro afirmar que esses dados não são úteis para analise